<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#JWST-Pipeline-Validation-Testing-Notebook:-Spec2,-fringe,-MIRI" data-toc-modified-id="JWST-Pipeline-Validation-Testing-Notebook:-Spec2,-fringe,-MIRI-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>JWST Pipeline Validation Testing Notebook: Spec2, fringe, MIRI</a></span><ul class="toc-item"><li><span><a href="#Introduction-and-summary-of-test-being-run" data-toc-modified-id="Introduction-and-summary-of-test-being-run-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction and summary of test being run</a></span></li><li><span><a href="#Documentation" data-toc-modified-id="Documentation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Documentation</a></span></li><li><span><a href="#Data-used" data-toc-modified-id="Data-used-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Data used</a></span><ul class="toc-item"><li><span><a href="#Create-temporary-directory" data-toc-modified-id="Create-temporary-directory-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Create temporary directory</a></span></li><li><span><a href="#Set-up-import-statements" data-toc-modified-id="Set-up-import-statements-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Set up import statements</a></span></li><li><span><a href="#Print-pipeline-version-number" data-toc-modified-id="Print-pipeline-version-number-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Print pipeline version number</a></span></li><li><span><a href="#Read-in-data-from-artifactory" data-toc-modified-id="Read-in-data-from-artifactory-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Read in data from artifactory</a></span></li><li><span><a href="#Read-in-input-image-as-JWST-data-model" data-toc-modified-id="Read-in-input-image-as-JWST-data-model-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Read in input image as JWST data model</a></span></li><li><span><a href="#Run-output-of-calwebb_detector1-through-calwebb_spec2" data-toc-modified-id="Run-output-of-calwebb_detector1-through-calwebb_spec2-1.3.6"><span class="toc-item-num">1.3.6&nbsp;&nbsp;</span>Run output of calwebb_detector1 through calwebb_spec2</a></span></li><li><span><a href="#Create-image-models-of-rate-and-cal-files" data-toc-modified-id="Create-image-models-of-rate-and-cal-files-1.3.7"><span class="toc-item-num">1.3.7&nbsp;&nbsp;</span>Create image models of rate and cal files</a></span></li><li><span><a href="#Get-flat_field-reference-file" data-toc-modified-id="Get-flat_field-reference-file-1.3.8"><span class="toc-item-num">1.3.8&nbsp;&nbsp;</span>Get flat_field reference file</a></span></li><li><span><a href="#Create-image-lists-for-looping" data-toc-modified-id="Create-image-lists-for-looping-1.3.9"><span class="toc-item-num">1.3.9&nbsp;&nbsp;</span>Create image lists for looping</a></span></li><li><span><a href="#Check-DQ-flagging" data-toc-modified-id="Check-DQ-flagging-1.3.10"><span class="toc-item-num">1.3.10&nbsp;&nbsp;</span>Check DQ flagging</a></span></li><li><span><a href="#Calculate-the-rate/cal-image-ratio" data-toc-modified-id="Calculate-the-rate/cal-image-ratio-1.3.11"><span class="toc-item-num">1.3.11&nbsp;&nbsp;</span>Calculate the rate/cal image ratio</a></span></li><li><span><a href="#Compare-flat-field-reference-file-with-the-rate/cal-image-ratio-and-check-that-they-are-equal" data-toc-modified-id="Compare-flat-field-reference-file-with-the-rate/cal-image-ratio-and-check-that-they-are-equal-1.3.12"><span class="toc-item-num">1.3.12&nbsp;&nbsp;</span>Compare flat field reference file with the rate/cal image ratio and check that they are equal</a></span></li></ul></li><li><span><a href="#About-this-Notebook" data-toc-modified-id="About-this-Notebook-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>About this Notebook</a></span></li></ul></li></ul></div>

<a id=“title_ID”></a>
# JWST Pipeline Validation Testing Notebook: Spec2, fringe, MIRI
Instruments Affected: MIRI

## Introduction and summary of test being run

This notebook processes an image through calwebb_spec2 and examines the output of the fringe
step. The steps are as follow:

1) Set up data path and directory and image file names.

2) Run output of calwebb_detector1 through the fringe step in calwebb_spec2.

3) Get fringe reference file from CRDS. 

4) Compare the fringe reference file with the rate/cal image ratio and check that they are the same.

## Documentation

The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

## Data used 

The data used in this test is a simulated MIRI image created using MIRISim. The documentation for MIRISim can be found here: https://wiki.miricle.org/bin/view/Public/MIRISim_Public?

Author: Mike Engesser (Adapted from Tea Temim's calwebb-image1 notebook)

### Create temporary directory

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

<TemporaryDirectory '/var/folders/v1/hp5rmzsx5kzgr8md63p7ptfh0002br/T/tmpmmanc7va'>


### Set up import statements

In [2]:
from astropy.io import fits, ascii
import pytest
import numpy as np
import numpy.ma as ma

import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.datamodels import RampModel, ImageModel, dqflags
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_spec2

from ci_watson.artifactory_helpers import get_bigdata
import crds
import os

2021-08-30 18:08:15,287 - stpipe - WARNING - /Users/mengesser/miniconda3/envs/jwst_validation_notebooks/lib/python3.8/site-packages/photutils/detection/findstars.py:33: AstropyDeprecationWarning: _StarFinderKernel was moved to the photutils.detection._utils module. Please update your import statement.
  warnings.warn(f'{name} was moved to the {deprecated[name]} module. '



In [3]:
# Specify CRDS locations and pmap
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

### Print pipeline version number

In [4]:
jwst.__version__ 

'1.1.0'

### Read in data from artifactory

In [5]:
file1 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits')
file2 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12MEDIUMexp1_rate.fits')
file3 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12SHORTexp1_rate.fits')
file4 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFULONG_34LONGexp1_rate.fits')
file5 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFULONG_34MEDIUMexp1_rate.fits')
file6 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFULONG_34SHORTexp1_rate.fits')

### Read in input image as JWST data model

In [6]:
im1 = ImageModel(file1)
im2 = ImageModel(file2)
im3 = ImageModel(file3)
im4 = ImageModel(file4)
im5 = ImageModel(file5)
im6 = ImageModel(file6)

### Run output of calwebb_detector1 through calwebb_spec2

In [7]:
def run_spec2(im):
    """Creates and runs an instance of the JWST spec2 pipeline. """
    
    spec2 = calwebb_spec2.Spec2Pipeline()
    spec2.bkg_subtract.skip = True
    spec2.assign_wcs.skip = False
    spec2.imprint_subtract.skip = True
    spec2.msa_flagging.skip = True
    spec2.extract_2d.skip = True
    spec2.wavecorr.skip = True
    spec2.srctype.skip = True
    spec2.straylight.skip = True
    spec2.fringe.skip = False
    spec2.pathloss.skip = True
    spec2.barshadow.skip = True
    spec2.photom.skip = True
    spec2.resample_spec.skip = True
    spec2.cube_build.skip = True
    spec2.extract_1d.skip = True
    spec2.flat_field.skip = True
    spec2.save_results = True
    
    spec2.run(im)

In [8]:
run_spec2(im1)
run_spec2(im2)
run_spec2(im3)
run_spec2(im4)
run_spec2(im5)
run_spec2(im6)

2021-08-30 18:08:50,805 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-08-30 18:08:50,806 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-08-30 18:08:50,809 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-08-30 18:08:50,810 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-08-30 18:08:50,812 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-08-30 18:08:50,814 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-08-30 18:08:50,818 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-08-30 18:08:50,820 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-08-30 18:08:50,821 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-08-30 18:08:50,823 - stpipe.Spec2Pipeline.master_

2021-08-30 18:08:50,964 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits' reftypes = ['camera', 'collimator', 'disperser', 'distortion', 'filteroffset', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'regions', 'specwcs', 'wavelengthrange']
2021-08-30 18:08:56,778 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2021-08-30 18:08:56,779 - stpipe.Spec2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2021-08-30 18:08:56,781 - stpipe.Spec2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2021-08-30 18:08:56,782 - stpipe.Spec2Pipeline - INFO - Prefetch for DISTORTION reference file is '/grp/crds/cache/references/jwst/jwst_miri_distortion_0029.asdf'.
2021-08-30 18:08:56,819 - stpipe.Spec2Pipeline - INFO - Prefetch for FILTEROFFSET reference file is 'N/A'.
2021-08-30 18:08:56,820 - stpipe.Spec2Pipeline - INFO - Prefetch for FORE refe

2021-08-30 18:09:44,014 - stpipe.Spec2Pipeline.flat_field - INFO - Step flat_field done
2021-08-30 18:09:44,138 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits>,).
2021-08-30 18:09:44,140 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'method': 'ModShepard', 'roi': 50, 'power': 1.0}
2021-08-30 18:09:44,145 - stpipe.Spec2Pipeline.straylight - INFO - Step skipped.
2021-08-30 18:09:44,147 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight done
2021-08-30 18:09:44,266 - stpipe.Spec2Pipeline.fringe - INFO - Step fringe running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIFUSHORT_12LONGexp1_

2021-08-30 18:09:53,309 - stpipe.Spec2Pipeline.fringe - INFO - FringeStep instance created.
2021-08-30 18:09:53,310 - stpipe.Spec2Pipeline.pathloss - INFO - PathLossStep instance created.
2021-08-30 18:09:53,312 - stpipe.Spec2Pipeline.barshadow - INFO - BarShadowStep instance created.
2021-08-30 18:09:53,314 - stpipe.Spec2Pipeline.photom - INFO - PhotomStep instance created.
2021-08-30 18:09:53,315 - stpipe.Spec2Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2021-08-30 18:09:53,317 - stpipe.Spec2Pipeline.cube_build - INFO - CubeBuildStep instance created.
2021-08-30 18:09:53,319 - stpipe.Spec2Pipeline.extract_1d - INFO - Extract1dStep instance created.
2021-08-30 18:09:53,460 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIFUSHORT_12MEDIUMexp1_rate.fits>,).
2021-08-30 18:09:53,479 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': 

2021-08-30 18:09:53,491 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIFUSHORT_12MEDIUMexp1_rate.fits' reftypes = ['camera', 'collimator', 'disperser', 'distortion', 'filteroffset', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'regions', 'specwcs', 'wavelengthrange']
2021-08-30 18:09:53,776 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2021-08-30 18:09:53,777 - stpipe.Spec2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2021-08-30 18:09:53,778 - stpipe.Spec2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2021-08-30 18:09:53,779 - stpipe.Spec2Pipeline - INFO - Prefetch for DISTORTION reference file is '/grp/crds/cache/references/jwst/jwst_miri_distortion_0030.asdf'.
2021-08-30 18:09:53,812 - stpipe.Spec2Pipeline - INFO - Prefetch for FILTEROFFSET reference file is 'N/A'.
2021-08-30 18:09:53,813 - stpipe.Spec2Pipeline - INFO - Prefetch for FORE re

...

2021-08-30 18:12:20,493 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIFULONG_34MEDIUMexp1_rate.fits' reftypes = ['camera', 'collimator', 'disperser', 'distortion', 'filteroffset', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'regions', 'specwcs', 'wavelengthrange']
2021-08-30 18:12:20,634 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2021-08-30 18:12:20,635 - stpipe.Spec2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2021-08-30 18:12:20,636 - stpipe.Spec2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2021-08-30 18:12:20,637 - stpipe.Spec2Pipeline - INFO - Prefetch for DISTORTION reference file is '/grp/crds/cache/references/jwst/jwst_miri_distortion_0033.asdf'.
2021-08-30 18:12:20,669 - stpipe.Spec2Pipeline - INFO - Prefetch for FILTEROFFSET reference file is 'N/A'.
2021-08-30 18:12:20,670 - stpipe.Spec2Pipeline - INFO - Prefetch for FORE ref

2021-08-30 18:12:56,274 - stpipe.Spec2Pipeline.flat_field - INFO - Step flat_field done
2021-08-30 18:12:56,370 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIFULONG_34MEDIUMexp1_rate.fits>,).
2021-08-30 18:12:56,373 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'method': 'ModShepard', 'roi': 50, 'power': 1.0}
2021-08-30 18:12:56,374 - stpipe.Spec2Pipeline.straylight - INFO - Step skipped.
2021-08-30 18:12:56,375 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight done
2021-08-30 18:12:56,473 - stpipe.Spec2Pipeline.fringe - INFO - Step fringe running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIFULONG_34MEDIUMexp

2021-08-30 18:13:03,276 - stpipe.Spec2Pipeline.fringe - INFO - FringeStep instance created.
2021-08-30 18:13:03,278 - stpipe.Spec2Pipeline.pathloss - INFO - PathLossStep instance created.
2021-08-30 18:13:03,279 - stpipe.Spec2Pipeline.barshadow - INFO - BarShadowStep instance created.
2021-08-30 18:13:03,281 - stpipe.Spec2Pipeline.photom - INFO - PhotomStep instance created.
2021-08-30 18:13:03,282 - stpipe.Spec2Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2021-08-30 18:13:03,284 - stpipe.Spec2Pipeline.cube_build - INFO - CubeBuildStep instance created.
2021-08-30 18:13:03,286 - stpipe.Spec2Pipeline.extract_1d - INFO - Extract1dStep instance created.
2021-08-30 18:13:03,424 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIFULONG_34SHORTexp1_rate.fits>,).
2021-08-30 18:13:03,443 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': No

2021-08-30 18:13:03,452 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIFULONG_34SHORTexp1_rate.fits' reftypes = ['camera', 'collimator', 'disperser', 'distortion', 'filteroffset', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'regions', 'specwcs', 'wavelengthrange']
2021-08-30 18:13:03,542 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2021-08-30 18:13:03,543 - stpipe.Spec2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2021-08-30 18:13:03,543 - stpipe.Spec2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2021-08-30 18:13:03,544 - stpipe.Spec2Pipeline - INFO - Prefetch for DISTORTION reference file is '/grp/crds/cache/references/jwst/jwst_miri_distortion_0034.asdf'.
2021-08-30 18:13:03,579 - stpipe.Spec2Pipeline - INFO - Prefetch for FILTEROFFSET reference file is 'N/A'.
2021-08-30 18:13:03,580 - stpipe.Spec2Pipeline - INFO - Prefetch for FORE refe

2021-08-30 18:13:40,879 - stpipe.Spec2Pipeline.flat_field - INFO - Step flat_field done
2021-08-30 18:13:40,973 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIFULONG_34SHORTexp1_rate.fits>,).
2021-08-30 18:13:40,976 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'method': 'ModShepard', 'roi': 50, 'power': 1.0}
2021-08-30 18:13:40,977 - stpipe.Spec2Pipeline.straylight - INFO - Step skipped.
2021-08-30 18:13:40,979 - stpipe.Spec2Pipeline.straylight - INFO - Step straylight done
2021-08-30 18:13:41,096 - stpipe.Spec2Pipeline.fringe - INFO - Step fringe running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIFULONG_34SHORTexp1_

### Create image models of rate and cal files

In [9]:
# Get names of cal files
input_file1 = file1.replace('rate.fits', 'cal.fits')
input_file2 = file2.replace('rate.fits', 'cal.fits')
input_file3 = file3.replace('rate.fits', 'cal.fits')
input_file4 = file4.replace('rate.fits', 'cal.fits')
input_file5 = file5.replace('rate.fits', 'cal.fits')
input_file6 = file6.replace('rate.fits', 'cal.fits')

In [10]:
# create image models of rate and cal files
im_cal1 = ImageModel(input_file1)
im_cal2 = ImageModel(input_file2)
im_cal3 = ImageModel(input_file3)
im_cal4 = ImageModel(input_file4)
im_cal5 = ImageModel(input_file5)
im_cal6 = ImageModel(input_file6)

im_rate1 = ImageModel(file1)
im_rate2 = ImageModel(file2)
im_rate3 = ImageModel(file3)
im_rate4 = ImageModel(file4)
im_rate5 = ImageModel(file5)
im_rate6 = ImageModel(file6)

### Get flat_field reference file

In [11]:
fringereffile1 = im_cal1.meta.ref_file.fringe.name
fringereffile2 = im_cal2.meta.ref_file.fringe.name
fringereffile3 = im_cal3.meta.ref_file.fringe.name
fringereffile4 = im_cal4.meta.ref_file.fringe.name
fringereffile5 = im_cal5.meta.ref_file.fringe.name
fringereffile6 = im_cal6.meta.ref_file.fringe.name

print('fringe reference file', fringereffile1)
print('fringe reference file', fringereffile2)
print('fringe reference file', fringereffile3)
print('fringe reference file', fringereffile4)
print('fringe reference file', fringereffile5)
print('fringe reference file', fringereffile6)
# find location of file
basename1 = crds.core.config.pop_crds_uri(fringereffile1)
basename2 = crds.core.config.pop_crds_uri(fringereffile2)
basename3 = crds.core.config.pop_crds_uri(fringereffile3)
basename4 = crds.core.config.pop_crds_uri(fringereffile4)
basename5 = crds.core.config.pop_crds_uri(fringereffile5)
basename6 = crds.core.config.pop_crds_uri(fringereffile6)

path1 = crds.locate_file(basename1, "jwst")
path2 = crds.locate_file(basename2, "jwst")
path3 = crds.locate_file(basename3, "jwst")
path4 = crds.locate_file(basename4, "jwst")
path5 = crds.locate_file(basename5, "jwst")
path6 = crds.locate_file(basename6, "jwst")
# open reference file
fringe_im1 = ImageModel(path1)
fringe_im2 = ImageModel(path2)
fringe_im3 = ImageModel(path3)
fringe_im4 = ImageModel(path4)
fringe_im5 = ImageModel(path5)
fringe_im6 = ImageModel(path6)

2021-08-30 18:13:55,066 - stpipe - WARNING - /Users/mengesser/miniconda3/envs/jwst_validation_notebooks/lib/python3.8/site-packages/stdatamodels/validate.py:34: ValidationWarning: While validating meta.exposure.type the following error occurred:
'MIR_FLAT-MRS' is not one of ['FGS_DARK', 'FGS_FOCUS', 'FGS_IMAGE', 'FGS_INTFLAT', 'FGS_SKYFLAT', 'FGS_ACQ1', 'FGS_ACQ2', 'FGS_FINEGUIDE', 'FGS_ID-IMAGE', 'FGS_ID-STACK', 'FGS_TRACK', 'MIR_4QPM', 'MIR_CORONCAL', 'MIR_DARKALL', 'MIR_DARKIMG', 'MIR_DARKMRS', 'MIR_FLATIMAGE', 'MIR_FLATIMAGE-EXT', 'MIR_FLATMRS', 'MIR_FLATMRS-EXT', 'MIR_IMAGE', 'MIR_LRS-FIXEDSLIT', 'MIR_LRS-SLITLESS', 'MIR_LYOT', 'MIR_MRS', 'MIR_TACONFIRM', 'MIR_TACQ', 'NIS_AMI', 'NIS_DARK', 'NIS_EXTCAL', 'NIS_FOCUS', 'NIS_IMAGE', 'NIS_LAMP', 'NIS_SOSS', 'NIS_TACQ', 'NIS_TACONFIRM', 'NIS_WFSS', 'NRC_CORON', 'NRC_DARK', 'NRC_FLAT', 'NRC_FOCUS', 'NRC_GRISM', 'NRC_IMAGE', 'NRC_WFSS', 'NRC_LED', 'NRC_WFSC', 'NRC_TACONFIRM', 'NRC_TACQ', 'NRC_TSGRISM', 'NRC_TSIMAGE', 'NRS_AUTOFLAT', 'NRS_

fringe reference file crds://jwst_miri_fringe_0049.fits
fringe reference file crds://jwst_miri_fringe_0034.fits
fringe reference file crds://jwst_miri_fringe_0046.fits
fringe reference file crds://jwst_miri_fringe_0045.fits
fringe reference file crds://jwst_miri_fringe_0044.fits
fringe reference file crds://jwst_miri_fringe_0042.fits


### Create image lists for looping

In [12]:
file_list = [file1, file2, file3, file4, file5, file6]
im_list = [im_cal1, im_cal2, im_cal3, im_cal4, im_cal5, im_cal6]
rate_list = [im_rate1, im_rate2, im_rate3, im_rate4, im_rate5, im_rate6]
fringe_list = [fringe_im1, fringe_im2, fringe_im3, fringe_im4, fringe_im5, fringe_im6]

### Check DQ flagging 
Any pixel flagged as NON_SCIENCE should also be flagged as DO_NOT_USE. Check if this is in place in both the input reference file and for the output science file of the calwebb_image2 pipeline. If there are no assert errors, the test below passes.

In [13]:
# Check if the output cal file is flagged properly
# return DQ arrays for masking
def check_dq_flags(im_cal, fringe_im):
    # Test that all pixels flagged with NON_SCIENCE are also flagged as DO_NOT_USE
    nonsciencearray = (im_cal.dq & dqflags.pixel['NON_SCIENCE'] > 0)
    badarray = (im_cal.dq & dqflags.pixel['DO_NOT_USE'] > 0)
    assert nonsciencearray.all() == badarray.all()

    # Test if the input reference file had the flags all set the same way
    nonsciencearray = (fringe_im.dq & dqflags.pixel['NON_SCIENCE'] > 0)
    badarray = (fringe_im.dq & dqflags.pixel['DO_NOT_USE'] > 0)
    assert nonsciencearray.all() == badarray.all()
    
    return badarray

badarrays = [check_dq_flags(im, fringe) for im,fringe in zip(im_list, fringe_list)]

### Calculate the rate/cal image ratio

In [19]:
def get_ratios(im, rate, mask):
    # compute and return the ratio of the rate and cal images
    im_mask = ma.masked_array(im.data, mask=mask)
    #ratio = im.data/rate_mask
    ratio = rate.data/im_mask.data

    return ratio

ratios = [get_ratios(cal_im, rate_im, badarray) for cal_im, rate_im, badarray in zip(im_list, rate_list, badarrays)]

2021-08-30 18:22:27,132 - stpipe - WARNING - <ipython-input-19-2f25071d3d77>:5: RuntimeWarning: invalid value encountered in true_divide
  ratio = rate.data/im_mask.data



### Compare flat field reference file with the rate/cal image ratio and check that they are equal

The minimum and maximum values should be 1.0, implying all values are 1, and the images are equal.

In [20]:
def check_fringe(ratio_im, fringe_im):
    return ratio_im/fringe_im.data

check_fringes = [check_fringe(ratio, fringe) for ratio, fringe in zip(ratios, fringe_list)]

In [25]:
def check_min_max(check_fringe_im, im_name):
    print('Minimum and maximum values for {} image are: {:.2e} and {:.2e}'.format(im_name, np.nanmin(check_fringe_im), np.nanmax(check_fringe_im)))
    #assert np.nanmin(check_fringe_im) == 1.0 and np.nanmax(check_fringe_im) == 1.0

for im, name in zip(check_fringes, file_list):
    im_name = name.split('/')[-1]
    check_min_max(im, im_name)

Minimum and maximum values for det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits image are: 1.00e+00 and 1.00e+00
Minimum and maximum values for det_image_seq1_MIRIFUSHORT_12MEDIUMexp1_rate.fits image are: 1.00e+00 and 1.00e+00
Minimum and maximum values for det_image_seq1_MIRIFUSHORT_12SHORTexp1_rate.fits image are: 1.00e+00 and 1.00e+00
Minimum and maximum values for det_image_seq1_MIRIFULONG_34LONGexp1_rate.fits image are: 1.00e+00 and 1.00e+00
Minimum and maximum values for det_image_seq1_MIRIFULONG_34MEDIUMexp1_rate.fits image are: 1.00e+00 and 1.00e+00
Minimum and maximum values for det_image_seq1_MIRIFULONG_34SHORTexp1_rate.fits image are: 1.00e+00 and 1.00e+00


<a id="about_ID"></a>
## About this Notebook
**Author:** Mike Engesser, Science Support Analyst I, INS
<br>**Updated On:** 08/30/2021